<a href="https://colab.research.google.com/github/Voldemort-vk/SOC-25/blob/main/SOC_25_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading Datasets

In [ ]:
!pip install "deeplake<4"

In [ ]:
import deeplake
train_set = deeplake.load('hub://activeloop/fer2013-train')
public_test_set = deeplake.load('hub://activeloop/fer2013-public-test')
private_test_set = deeplake.load('hub://activeloop/fer2013-private-test')

In [3]:
len(train_set)

28709

In [4]:
len(public_test_set)

3589

In [5]:
len(private_test_set)

3589

# Importing Libraries

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [ ]:
train_set[0]

In [8]:
train_set_images = train_set['images'].numpy()
train_set_label = train_set['labels'].numpy()
public_test_set_images = (public_test_set['images'].numpy().astype(np.float32)/(255.0)).reshape(len(public_test_set), 48, 48, 1)
public_test_set_label = public_test_set['labels'].numpy()
private_test_set_images = (private_test_set['images'].numpy().astype(np.float32)/(255.0)).reshape(len(private_test_set), 48, 48, 1)
private_test_set_label = private_test_set['labels'].numpy()

In [9]:
expressions = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

In [ ]:
index = np.random.randint(0, len(train_set))
image = train_set_images[index]
label = train_set_label[index][0]
plt.imshow(image, cmap = 'gray')
print('Label: ', label)
print('Expression: ', expressions[label])

# Splitting the training dataset into train, validation, and test sets
(also normalizing the pixel values to the range [0, 1])

In [14]:
idxs = np.random.permutation(len(train_set))

In [15]:
#len(train_set) = 28709
#Validation = 2871
#Test = 2871
#Train = 22967

In [16]:
validation_idx = idxs[:2871]
test_idx = idxs[2871:5742]
train_idx = idxs[5742:]

In [17]:
Validation_image = (train_set_images[validation_idx].astype(np.float32)/(255.0)).reshape(len(validation_idx), 48, 48, 1)
Validation_label = train_set_label[validation_idx]
Test_image = (train_set_images[test_idx].astype(np.float32)/(255.0)).reshape(len(test_idx), 48, 48, 1)
Test_label = train_set_label[test_idx]
Train_image = (train_set_images[train_idx].astype(np.float32)/(255.0)).reshape(len(train_idx), 48, 48, 1)
Train_label = train_set_label[train_idx]

# Validation set images


In [ ]:
index = np.random.randint(0, len(validation_idx))
image = Validation_image[index]
label = Validation_label[index][0]
plt.imshow(image, cmap = 'gray')
print('Label: ', label)
print('Expression: ', expressions[label])

# Test set images


In [ ]:
index = np.random.randint(0, len(test_idx))
image = Test_image[index]
label = Test_label[index][0]
plt.imshow(image, cmap = 'gray')
print('Label: ', label)
print('Expression: ', expressions[label])

# Train set images

In [ ]:
index = np.random.randint(0, len(train_idx))
image = Train_image[index]
label = Train_label[index][0]
plt.imshow(image, cmap = 'gray')
print('Label: ', label)
print('Expression: ', expressions[label])

#Model


In [ ]:
model = Sequential([
    Conv2D(32, (3, 3),  input_shape = (48, 48, 1), activation = 'relu', padding = 'same'),
    MaxPooling2D((2, 2)),
    Dropout(0.2),

    Conv2D(64, (3, 3), activation = 'relu', padding = 'same'),
    MaxPooling2D((2, 2)),
    Dropout(0.2),

    Conv2D(128, (3, 3), activation = 'relu', padding = 'same'),
    MaxPooling2D((2, 2)),
    Dropout(0.2),

    Flatten(),
    Dense(128, activation = 'relu'),
    Dense(7, activation = 'softmax')
])

In [22]:
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])

# Training the model on our train set


In [ ]:
model.fit(Train_image, Train_label, epochs = 5, batch_size = 100)

# Evaluating the model on our Validation set

In [ ]:
model.evaluate(Validation_image, Validation_label, batch_size = 1)

# Evaluating the model on our Test set

In [ ]:
model.evaluate(Test_image, Test_label, batch_size = 1)

# Evaluating the model on the given public test set

In [ ]:
model.evaluate(public_test_set_images, public_test_set_label, batch_size = 1)

# Evaluating the model on the given private test set

In [ ]:
model.evaluate(private_test_set_images, private_test_set_label, batch_size = 1)

# Comparing actual and predicted results on the given Public test set

In [ ]:
index = np.random.randint(0, len(public_test_set))
image = public_test_set_images[index].reshape(1, 48, 48, 1)
actual_label = public_test_set_label[index]
prediction = model.predict(image)
predicted_label = (np.argmax(prediction))

plt.imshow(image.reshape(48, 48, 1), cmap='gray')
print('Actual Label: ', actual_label[0])
print('Predicted Label: ', predicted_label)
print('Actual Expression: ', expressions[actual_label[0]])
print('Predicted Expression: ', expressions[predicted_label])

# Comparing actual and predicted results on the given Private test set

In [ ]:
index = np.random.randint(0, len(public_test_set))
image = private_test_set_images[index].reshape(1, 48, 48, 1)
actual_label = private_test_set_label[index]
prediction = model.predict(image)
predicted_label = (np.argmax(prediction))

plt.imshow(image.reshape(48, 48, 1), cmap='gray')
print('Actual Label: ', actual_label[0])
print('Predicted Label: ', predicted_label)
print('Actual Expression: ', expressions[actual_label[0]])
print('Predicted Expression: ', expressions[predicted_label])